In [ ]:
#Import libraries
import math
import os
import re
import csv
import sys
import io
import hashlib

import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from decimal import Decimal
from requests import get
from zipfile import ZipFile
import shutil
import wget
import re
import tensorflow as tf
#tf.enable_eager_execution()

from pandas.io import sql
import pyodbc
from sqlalchemy import create_engine

import codecs
import seaborn as sns
from dask import dataframe as df1

##Import section to ensure the main directory is in the path
#module_path = os.path.abspath(os.path.join('..'))
#if module_path not in sys.path:
#    sys.path.append(module_path)

import warnings
warnings.filterwarnings('ignore')
sys.path.append('../utils/')
from sqlalchemy import create_engine
import datetime as dt

print('Python: ' + sys.version.split('|')[0])
print('Pandas: ' + pd.__version__)
print('pyODBC: ' + pyodbc.version)

In [ ]:
#setup SQL server config
DB = {'servername': 'DESKTOP-FPGABNM',
      'database': 'PAC_Data',
      'driver': 'driver=SQL Server Native Client 11.0'}

# create the connection
engine = create_engine('mssql+pyodbc://' + DB['servername'] + '/' + DB['database'] + "?" + DB['driver'])

In [ ]:
#Unzip early voting files into temp folder
zip_output_dir = r"..\\output\\temp\\"


for file in os.listdir("..\\data\\completion_zip"):
    zip_file = "..\\data\\earlyvoting\\" + file

    with ZipFile(zip_file) as zip_file:
        for member in zip_file.namelist():
            filename = os.path.basename(member)
            # skip directories
            if not filename:
                continue

            # copy file (taken from zipfile's extract)
            source = zip_file.open(member)
            target = open(os.path.join(zip_output_dir, filename), "wb")
            with source, target:
                shutil.copyfileobj(source, target)
                
## early voting testing
ev_file1 = open("\\data\\earlyvoting\\" + "ECTOR_EV_5_1_2023.xlsx", 'r', newline='', encoding='utf-8')
ev_file2 = open("\\data\\earlyvoting\\" + "ECTOR_EV_5_25_2023.xlsx", 'r', newline='', encoding='utf-8')

#
ectorcad_tax_roll = open("\\data\\ector_appraisal\\" + "Rpt_Acct_Detail.csv", 'r', newline=None)
#ectorcad_geodata = "\\data\\ector_appraisal_geodata\\Ectorparcel_GEODB.gdb\\" + "a0000000a.spx"

In [ ]:
##TO-DO code from well decline and spacing. Integrate into utils

"""Label map utility functions."""

import logging

import tensorflow as tf
from google.protobuf import text_format
from protos import string_int_label_map_pb2


def _validate_label_map(label_map):
  """Checks if a label map is valid.

  Args:
    label_map: StringIntLabelMap to validate.

  Raises:
    ValueError: if label map is invalid.
  """
  for item in label_map.item:
    if item.id < 1:
      raise ValueError('Label map ids should be >= 1.')


def create_category_index(categories):
  """Creates dictionary of COCO compatible categories keyed by category id.

  Args:
    categories: a list of dicts, each of which has the following keys:
      'id': (required) an integer id uniquely identifying this category.
      'name': (required) string representing category name
        e.g., 'cat', 'dog', 'pizza'.

  Returns:
    category_index: a dict containing the same entries as categories, but keyed
      by the 'id' field of each category.
  """
  category_index = {}
  for cat in categories:
    category_index[cat['id']] = cat
  return category_index


def convert_label_map_to_categories(label_map,
                                    max_num_classes,
                                    use_display_name=True):
  """Loads label map proto and returns categories list compatible with eval.

  This function loads a label map and returns a list of dicts, each of which
  has the following keys:
    'id': (required) an integer id uniquely identifying this category.
    'name': (required) string representing category name
      e.g., 'cat', 'dog', 'pizza'.
  We only allow class into the list if its id-label_id_offset is
  between 0 (inclusive) and max_num_classes (exclusive).
  If there are several items mapping to the same id in the label map,
  we will only keep the first one in the categories list.

  Args:
    label_map: a StringIntLabelMapProto or None.  If None, a default categories
      list is created with max_num_classes categories.
    max_num_classes: maximum number of (consecutive) label indices to include.
    use_display_name: (boolean) choose whether to load 'display_name' field
      as category name.  If False or if the display_name field does not exist,
      uses 'name' field as category names instead.
  Returns:
    categories: a list of dictionaries representing all possible categories.
  """
  categories = []
  list_of_ids_already_added = []
  if not label_map:
    label_id_offset = 1
    for class_id in range(max_num_classes):
      categories.append({
          'id': class_id + label_id_offset,
          'name': 'category_{}'.format(class_id + label_id_offset)
      })
    return categories
  for item in label_map.item:
    if not 0 < item.id <= max_num_classes:
      logging.info('Ignore item %d since it falls outside of requested '
                   'label range.', item.id)
      continue
    if use_display_name and item.HasField('display_name'):
      name = item.display_name
    else:
      name = item.name
    if item.id not in list_of_ids_already_added:
      list_of_ids_already_added.append(item.id)
      categories.append({'id': item.id, 'name': name})
  return categories


def load_labelmap(path):
  """Loads label map proto.

  Args:
    path: path to StringIntLabelMap proto text file.
  Returns:
    a StringIntLabelMapProto
  """
  with tf.gfile.GFile(path, 'r') as fid: 
  # with tf.compat.v2.io.gfile.GFile(path, 'r') as fid: # use this line to run it with TensorFlow version 2.x
    label_map_string = fid.read()
    label_map = string_int_label_map_pb2.StringIntLabelMap()
    try:
      text_format.Merge(label_map_string, label_map)
    except text_format.ParseError:
      label_map.ParseFromString(label_map_string)
  _validate_label_map(label_map)
  return label_map


def get_label_map_dict(label_map_path, use_display_name=False):
  """Reads a label map and returns a dictionary of label names to id.

  Args:
    label_map_path: path to label_map.
    use_display_name: whether to use the label map items' display names as keys.

  Returns:
    A dictionary mapping label names to id.
  """
  label_map = load_labelmap(label_map_path)
  label_map_dict = {}
  for item in label_map.item:
    if use_display_name:
      label_map_dict[item.display_name] = item.id
    else:
      label_map_dict[item.name] = item.id
  return label_map_dict


def create_category_index_from_labelmap(label_map_path):
  """Reads a label map and returns a category index.

  Args:
    label_map_path: Path to `StringIntLabelMap` proto text file.

  Returns:
    A category index, which is a dictionary that maps integer ids to dicts
    containing categories, e.g.
    {1: {'id': 1, 'name': 'dog'}, 2: {'id': 2, 'name': 'cat'}, ...}
  """
  label_map = load_labelmap(label_map_path)
  max_num_classes = max(item.id for item in label_map.item)
  categories = convert_label_map_to_categories(label_map, max_num_classes)
  return create_category_index(categories)


def create_class_agnostic_category_index():
  """Creates a category index with a single `object` class."""
  return {1: {'id': 1, 'name': 'object'}}


In [ ]:
insert_list = []

#helpers need to move into utils folder after testing
def swap_production_dates_for_time_delta(wellDF):
    import pandas as pd

    # generate new column with time delta rather than production date
    voters_masterDF['Time Delta'] = '69 days 00:00:00'
    voters_masterDF['Time Delta'] = pd.to_timedelta(voters_masterDF['Time Delta'])

    for ID in set(voters_masterDF['SOS_VoterID']):
        startDate = '2023_11_7' ##Election day
        delta = voters_masterDF.loc[voters_masterDF['SOS_VoterID'] == ID, 'eligible_date'] - startDate
        voters_masterDF.loc[wellDF['SOS_VoterID'] == API, 'Time Delta'] = delta


    return voterDF

def yyyymmdd(date):
    date = decode_date(date)
    #Changes format YYYYMMDD from a series of numbers to datetime object
    try:
        val = datetime.strptime(date, '%Y%m%d').strftime('%m/%d/%Y')
    except ValueError:
        val = None
    return val
    
def yyyymm(yyyymm):
    yyyymm = decode_date(yyyymm)
    #Changes format YYYYMM from a series of numbers to datetime object
    #makes the date the first day of the month
    try:
        val = date(year=int(yyyymm[0:4]), month=int(yyyymm[4:]), day=1).strftime('%m/01/%Y')
    except ValueError:
        val = None

    return val

def datefix(date):
    date = decode_date(date)
    #Changes format YYYYMMDD from a series of numbers to datetime object
    try:
        val = datetime.strptime(date, '%Y%m%d').strftime('%Y-%m-01')
    except ValueError:
        val = None
    return val

def numeric(num):
    num = ebc_decode(num)
    try: ##using a try to ensure the values passed are actually 0-9 with no other characters
        val = int(num)
    except:
        val = None

    return val



voters_master_columns = ['SOS_VoterID', 'idnumber', 'voter_status', 'party_code', 'last_name', 'first_name', 'middle_name', 'name_suffix', 'street_number', 'street_building',
                      'street_pre_dir', 'street_name', 'street_type', 'unit_type', 'unit', 'city', 'zip', 'zip4', 'mail_1', 'mail_2', 'mail_3', 'mail_city',
                      'mail_state', 'mail_zip', 'mail_zip_4', 'sex', 'birth_date', 'eligible_date', 'effective_date', 'precinct', 'precsub', 'district_01', 'district_02',
                      'district_03', 'district_04', 'district_05', 'district_06', 'district_07', 'district_08', 'district_09', 'district_10', 'district_11', 'district_12',
                      'district_13', 'district_14', 'district_15', 'district_16', 'district_17', 'district_18', 'district_19', 'district_20', 'district_20', 'district_21',
                      'district_22', 'district_23', 'district_24', 'district_25', 'district_26', 'district_27', 'district_28', 'district_29', 'district_30', 'district_31',
                      'district_32', 'district_33', 'district_34', 'district_35', 'district_36', 'district_37', 'district_38', 'district_39', 'district_40', 'district_41',
                      'district_42', 'district_43', 'district_44', 'district_45', 'district_46', 'district_47', 'district_48', 'district_49', 'district_50',
                      'election_code1', 'vote_type1', 'party_code1', 'election_code2', 'vote_type2', 'party_code2', 'election_code3', 'vote_type3', 'party_code3',
                      'election_code4', 'vote_type4', 'party_code4', 'election_code5', 'vote_type5', 'party_code5', 'election_code6', 'vote_type6', 'party_code6',
                      'election_code7', 'vote_type7', 'party_code7', 'election_code8', 'vote_type8', 'party_code8', 'election_code9', 'vote_type9', 'party_code9',
                      'election_code10', 'vote_type10', 'party_code10', 'election_code11', 'vote_type11', 'party_code11', 'election_code12', 'vote_type12', 'party_code12', 
                      'election_code13', 'vote_type13', 'party_code13', 'election_code14', 'vote_type14', 'party_code14', 'election_code15', 'vote_type15', 'party_code15',
                      'election_code16', 'vote_type16', 'party_code16', 'election_code17', 'vote_type17', 'party_code17', 'election_code18', 'vote_type18', 'party_code18',
                      'election_code19', 'vote_type19', 'party_code19', 'election_code20', 'vote_type20', 'party_code20', 'election_code21', 'vote_type21', 'party_code21',
                      'election_code22', 'vote_type22', 'party_code22', 'election_code23', 'vote_type23', 'party_code23', 'election_code24', 'vote_type24', 'party_code24',
                      'election_code25', 'vote_type25', 'party_code25', 'election_code26', 'vote_type26', 'party_code26', 'election_code27', 'vote_type27', 'party_code27',
                      'election_code28', 'vote_type28', 'party_code28', 'election_code29', 'vote_type29', 'party_code29', 'election_code30', 'vote_type30', 'party_code30',
                      'election_code31', 'vote_type31', 'party_code31', 'election_code32', 'vote_type32', 'party_code32', 'election_code33', 'vote_type33', 'party_code33',
                      'election_code34', 'vote_type34', 'party_code34', 'election_code35', 'vote_type35', 'party_code35', 'election_code36', 'vote_type36', 'party_code36',
                      'election_code37', 'vote_type37', 'party_code37', 'election_code38', 'vote_type38', 'party_code38', 'election_code39', 'vote_type39', 'party_code39',
                      'election_code40', 'vote_type40', 'party_code40', 'registration_date', 'party_affiliation_date', 'last_activity_date', 'precinct_group', 'phone_number',
                      'ID_Compliant', 'Absentee_Category', 'Absentee_Category_Date', 'Ethnicity']
voters_master_header = pd.DataFrame(columns=voters_master_columns)


#cf1 and countidnumberpercf_1 transposed? wtf? consistent in test files... so far
early_voter_columns = [ 'idnumber', 'name', 'precinct', 'ballot_party', 'early_voting_site', 'precinct_sub', 'activity_date', 'voter_party', 'mailing', 'ballot_style',
                      'privacy_code', 'address', 'count_idnumber_per_cf_4', 'cf4', 'count_idnumber_per_cf_3', 'cf3', 'count_idnumber_per_cf_2', 'cf2', 'cf1', 'count_idnumber_per_cf_1',
                      'countidnumber_per_repot', 'cf_party_selection', 'cf_site_selected', 'cf_ddate_range', 'cf_reg_party_selection']
early_voter_header = pd.DataFrame(columns=early_voter_columns)

ector_appraisal_columns = [ 'appraisal_year', 'account_number', 'aay_gis_parcel_id', 'aay_division_cdx', 'aay_prop_type_cd', 'aon_master_owner_id', 'multi_own_ind', 
                      'owner_name', 'owner_name2', 'owner_name3', 'address_line1', 'address_line2', 'city', 'state', 'zip', 'undivided_int_pct', 'ctm_imp_sptd_nm',
                      'ctm_land_sptd_nm', 'ctm_prod_sptd_nm', 'LIF_TRANSFER_DT', 'LIF_TRANSFER_CDX', 'LIF_VOLUME_NUM', 'LIF_PAGE_NUM', 'LIF_INSTRUMENT_NUM'
                      'LIF_ABSTRACT_NUM', 'LIF_SUBDIVISION_NAME', 'LIF_DESC_LINE1', 'LIF_DESC_LINE2', 'LIF_DESC_LINE3', 'LIF_DESC_LINE4', 'LIF_DESC_LINE5',
                      'LIF_DESC_LINE6', 'LIF_DESC_LINE7', 'PROP_LOCATION', 'TOA_STREET_NUM', 'TOA_STREET_DIR', 'TOA_STREET_NAME', 'TOA_STREET_SUFFIX', 'AG_LAND_ACRES',
                      'LAND_ACRES', 'LAND_SF', 'LAND_FF', 'AAY_LAND_VAL', 'AAY_IMPR_VAL', 'AAY_PERS_VAL', 'AAY_MNRL_VAL', 'AAY_TOT_VAL', 'AAY_PROD_LOSS_VAL', 
                      'AAY_CAP_LOSS_VAL', 'AAY_APPR_VAL', 'ACV_LAND_MKT_VAL', 'ACV_PROD_MKT_VAL', 'ACV_PROD_VAL', 'HS_EX_IND', 'O65_EX_IND', 'DIS_EX_IND', 'DV1_EX_IND',
                      'DV2_EX_IND', 'ABT_EX_IND', 'HST_EX_IND', 'FPT_EX_IND', 'POL_EX_IND', 'TIF_EX_IND', 'SOL_EX_IND', 'WAT_EX_IND', 'TOT_EX_IND', 'PRO_EX_IND',
                      'TOT_EX_TYP_CD', 'AAY_CITY_JURIS_CD', 'AAY_COUNTY_JURIS_CD', 'AAY_ISD_JURIS_CD', 'AAY_HOSPITAL_JURIS_CD', 'AAY_COLLEGE_JURIS_CD', 'AAY_SPEC_JURIS_CD',
                      'AAY_SPEC2_JURIS_CD', 'AAY_SPEC3_JURIS_CD', 'AAY_SPEC4_JURIS_CD', 'AAY_SPEC5_JURIS_CD', 'CTM_CITY_JURIS_NM', 'CTM_HOSPITAL_JURIS_NM', 'CTM_SPEC2_JURIS_NM',
                      'CTM_SPEC3_JURIS_NM', 'CTM_SPEC4_JURIS_NM', 'CTM_SPEC5_JURIS_NM', 'CTM_SIC_CDX', 'CTM_NBHD_CDX', 'CTM_P_BLDG_CLASS_CDX', 'YEAR_BUILT', 'AREA_SIZE',
                      'VH1_EX_IND', 'LOW_EX_IND', 'GIT_EX_IND', 'PWR_EX_IND', 'AAY_MAPSCO_GRID', 'AAY_MAP_NUM', 'AAY_OVRLP_ACCOUNT_NUM', 'AAY_EXTRNL_ACCOUNT_NUM' ]
ector_appraisal_header = pd.DataFrame(columns=ector_appraisal_columns)



def clearList():
    insert_list.clear()

def insertDB_voterlist():
    df_ector_voters_master = pd.DataFrame(insert_list, columns=voters_master_columns)
    #print(df_w2_well_master)
    df_ector_voters_master['VOTER_DATE'] = pd.to_datetime(df_ector_voters_master['VOTER_DATE'], format='%Y%m', errors='coerce')



#limits for testing        
maxct=10
ct=0
ct2=0

for csvfile in os.listdir("..\\output\\temp"):
    #print(csvfile)
    csvlist.clear()
    reader = open("..\\output\\temp\\" + csvfile, 'rb', newline=None)
    data = reader.read().replace(b'\r\n', b'').splitlines()
    binary_file = open("..\\output\\temp\\" + csvfile, "w")
    for line in lines:
    if Limiting_Counter == True and ct > maxct:
        break
    #regex data extract by key **** testing
    if re.match(r'^.}(\d{2})}(\d{6})}2023}04', line):
        insert_list.append(line.rstrip('\n').replace(' ', '').split('}'))
        insertDB_voterlist()
    clearList()

In [ ]:
# Build correlation matrix for all gathered values

corr = data[['voter_status', 'party_code', 'precinct', 'days_registered']].corr()

plt.figure(figsize=(8, 4))
mask = np.triu(np.ones_like(corr, dtype=np.bool))
corr_plot = sns.heatmap(corr, mask=mask, annot=True, cmap='viridis')
corr_plot.set_title(f"Correlation matrix for {company_name}")

for item in corr_plot.get_xticklabels():
    item.set_rotation(45)

#plt.show()
plt.subplots_adjust(left=0.15,bottom=0.25)
plt.savefig('../results/Early_Voting_' + ev_date + 'Heatmap.png')

In [ ]:
#train tensorflow models for comments

features_considered = ['']
features = data[features_considered]

data_number = features.shape[0]
train_split = int(0.9 * data_number)

dataset = features.values
data_mean = dataset[:train_split].mean(axis=0)
data_std = dataset[:train_split].std(axis=0)

dataset = (dataset-data_mean)/data_std

print(f"mean normalized dataset of features = \n\n {dataset} \n")

print(f"shape of dataset = {dataset.shape}")

# batch generation algorithm

def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])

    return np.array(data), np.array(labels)

# define partitions

# INPUT: [to predict, past values used, prediction values, steps]

labels = dataset[:, 0] #  getting company close
n_past = 50 #  time stamps that will affect the future values (the more the more computing)
n_prediction = 5 #  the time stamp in the future of which we want to find the prediction
STEP = 1

train_partitions, train_labels = multivariate_data(dataset, labels, 0, train_split, n_past, n_prediction, STEP, single_step=False)

valid_partitions, valid_labels = multivariate_data(dataset, labels, train_split, None, n_past, n_prediction, STEP, single_step=False)